In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import time
import math

import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# START/END OF SENT TOKENS
SOS_token = 0
EOS_token = 1

# MAX SENTENCE LEN
MAX_LENGTH = 10

# HIDDEN DIM OF RNN
hidden_size = 256

# LIST OF PHRASES WE WANT SENTENCES TO START WITH
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


In [3]:
# USED TO DEFINE A LANGUAGE DICTIONARY W/ OTHER NECESSARY COMPONENTS
class Lang:
    def __init__(self, name):
        self.name = name
        # WORD INDEX REP, WORD COUNT, INDEX WORD REP
        self.word2index = {}
        self.word2count = {}
        # EVERY LANG NEEDS START/END TOKEN
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        # CHECK IF WORD IN DICT ALREADY, IF NOT INSERT
        # ELSE WE JUST UPDATE THE WORD COUNT
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

In [5]:
# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [6]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse input/output order, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [7]:
def filterPair(p):
    # REMOVE PAIRS WHERE INPUT/OUTPUT TOO LONG AND ONLY USE
    # THE START PHRASES WE WANTED 
    # print(p)
    return len(p[0].split('\t')) < MAX_LENGTH and \
        len(p[1].split('\t')) < MAX_LENGTH #\
        # and p[1].startswith(eng_prefixes)

In [8]:
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [9]:
def prepareData(lang1, lang2, reverse=False):
    # GET OUR LANGUAGE DICTS AND THE SENTENCES
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    # REMOVE TOO LONG SENTENCES, SENTENCES W/ INCORRECT START
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    # FILL OUR DICT
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [10]:
# GET OUR INPUT/OUTPUT DICT AND THE USABLE SENTENCES
# input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
# input_lang, output_lang, pairs = prepareData('guenya', 'eng', True)
input_lang, output_lang, pairs = prepareData('sindarin', 'eng', True)
# PRINT RANDOM SENTENCE EXAMPLE
print(random.choice(pairs))

Reading lines...
Read 2318 sentence pairs
Trimmed to 2318 sentence pairs
Counting words...
Counted words:
eng 755
sindarin 887
['give knowledge about', 'istantho o']
